In [ ]:
import pymysql
import pandas as pd
import getpass
import os
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import display, HTML
import json

## Set up our MySQL connection with SQL Alchemy (this helps us to read directly into Pandas DataFrames

In [ ]:
conn = pymysql.connect(host="35.233.174.193",port=3306,
                           user=getpass.getpass("Enter username for MIMIC2 database"),
                           passwd=getpass.getpass("Enter password for MIMIC2 database"),
                           db='mimic2')

## Before we move ahead, we will do some counts of patients, admissions and notes to ensure connectivity and also get a sense of the dataset

In [ ]:
display(pd.read_sql_query('SELECT count(*) as PatientCount from d_patients', conn))
display(pd.read_sql_query('SELECT count(*) as AdmissionCount from admissions', conn))
display(pd.read_sql_query('SELECT count(*) as NoteCount from noteevents', conn))

## MIMIC-II has tables for Admissions, ICD-9 codes, notes and many other pieces of data

In [ ]:
display(pd.read_sql_query('SELECT * from admissions LIMIT 5', conn))

In [ ]:
display(pd.read_sql_query('SELECT * from icd9 LIMIT 5', conn))

In [ ]:
display(pd.read_sql_query('SELECT * from noteevents LIMIT 5', conn))

In [ ]:
# now let's get a frame of Patient/Admit/Pneumonia
pneumonia_query = """
    SELECT 
a.subject_id
,a.hadm_id
,a.admit_dt
,(CASE WHEN pneu.HADM_ID IS NOT NULL THEN 1 ELSE 0 END) as Encounter_Pneumonia_Diagnosis
FROM admissions a
LEFT JOIN 
(
    SELECT
    d.HADM_ID
    FROM  icd9 d
    WHERE 
        (code like '486%%')
    GROUP BY d.HADM_ID
) pneu
ON a.HADM_ID = pneu.HADM_ID
"""
pat_admit_pneumonia_df = pd.read_sql_query(pneumonia_query, conn)
display(pat_admit_pneumonia_df)

In [ ]:
# let's get a count of how many PNEUMONIA vs NO-PNEUMONIA admits we have
pneumonia_admit_count_df = pat_admit_pneumonia_df.groupby('Encounter_Pneumonia_Diagnosis').size()
display(pneumonia_admit_count_df)

In [ ]:
# before pulling note text, let's get a distribution of how many RADIOLOGY reports
# typically exist per admission
visit_rad_report_count_query = """
SELECT
n.hadm_id
,count(*) as rad_note_count
FROM d_patients p
INNER JOIN noteevents n
    ON n.subject_id = p.subject_id
WHERE 
    Category = 'RADIOLOGY_REPORT' 
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND n.hadm_id IS NOT NULL
GROUP BY n.hadm_id
ORDER BY count(*) DESC
"""
visit_rad_report_count_df = pd.read_sql_query(visit_rad_report_count_query, conn)
display(visit_rad_report_count_df.head(10))

## Scipy had useful methods for describing distributions like our count of chest x-rays per encounter

In [ ]:
visit_rad_report_count_df['rad_note_count'].describe()


## Notes in MIMIC have a category (e.g. "RADIOLOGY_REPORT") and within the text there are often "sub categories" on the second line of the file.  Pulling the appropriate sub categories as a few "like" statements does the job, but it is worth looking at some of these on your own

In [ ]:
# before pulling note text, let's get a distribution of how many RADIOLOGY reports
# typically exist per admission
visit_rad_report_count_query = """
SELECT
n.hadm_id
,count(*) as rad_note_count
FROM d_patients p
INNER JOIN noteevents n
    ON n.subject_id = p.subject_id
WHERE 
    Category = 'RADIOLOGY_REPORT' 
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND n.hadm_id IS NOT NULL
GROUP BY n.hadm_id
ORDER BY count(*) DESC
"""
visit_rad_report_count_df = pd.read_sql_query(visit_rad_report_count_query, conn)
display(visit_rad_report_count_df)

## Some patients have only one radiology report but several have multiple.  This graph looks at that distribution

In [ ]:
rad_note_count_grouping = visit_rad_report_count_df.groupby('rad_note_count').size()
#display(rad_note_count_grouping)

note_count_bins = rad_note_count_grouping.index.values
#print(note_count_bins)
note_frequencies = rad_note_count_grouping.values
#print(note_frequencies)

fig = plt.figure(figsize=(16, 8))

plt.xlabel('Total Radiology Chest X-Ray Notes per visit')
plt.ylabel('Total Visits')

plt.bar(note_count_bins, note_frequencies)

## We can then can pull these notes into a frame

In [ ]:
# now let's pull a frame of all the FIRST (sorted by text which begins with date) CHEST X-RAY notes
chest_xray_note_query = """
SELECT
subject_id
,hadm_id
,charttime
,LTRIM(RTRIM(text)) as text
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND subject_id is not NULL
    AND hadm_id is not NULL
GROUP BY subject_id, hadm_id, charttime, text
"""
chest_xray_note_df = pd.read_sql_query(chest_xray_note_query, conn)
display(chest_xray_note_df.head(10))

## Much like a SQL "join" we can combine our frame which has ICD-9 codes with the frame that has notes so that we can sample from these intelligently

In [ ]:
pneumonia_note_df = pd.merge(pat_admit_pneumonia_df, chest_xray_note_df, on = ['subject_id', 'hadm_id'])
display(pneumonia_note_df.head(10))

## Saving Data in a Machine-Learning Friendly Manner

We've created an environment that allows us to read data from a database and, if we wanted, do NLP and machine learning on the data that we've extracted. But our data isn't quite in the format that most machine learning pipelines work, which tend to expect data to be in individual files per case organized in directories by category (e.g. pneumonia/no pneumonia).

So let's wrap up the notebook by writing the data in a machine-learning friendly manner.



In [ ]:
OUTDIR = "/Users/brian/GDrive"
pdir = os.path.join(OUTDIR, "MBSI", "pneumonia")
npdir = os.path.join(OUTDIR, "MBSI", "nopneumonia")

if not os.path.exists(pdir):
    os.makedirs(pdir)
    
if not os.path.exists(npdir):
    os.makedirs(npdir)

In [ ]:
# now let's list out some of the notes where Pneumonia WAS diagnosed
pneumonia_positive_notes = pneumonia_note_df[pneumonia_note_df['Encounter_Pneumonia_Diagnosis'] == 1]['text'].head(1).values


for index, row in pneumonia_note_df.iterrows():
    if row['Encounter_Pneumonia_Diagnosis'] == 1:
        odir = pdir
    else:
        odir = npdir
    with open(os.path.join(odir, "%05d.txt"%index), "w") as f:
        f.write(row['text'])
    with open(os.path.join(odir, "%05d.json"%index), "w") as f:
        meta = {"subject_id":row["subject_id"], 
                "hadm_id":row["hadm_id"],
                "admit_dt":row["admit_dt"].isoformat(),
                "charttime":row["charttime"].isoformat()}
        json.dump(meta, f)
    #sys.stdout.write(note)

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2017.<br/>
Presenters : Dr. Wendy Chapman, Jianlin Shi and Kelly Peterson